## DATA CLEANING

En este apartado se realizaran las etapas de preprocesamiento de datos.

Trabajaremos con con nuestra tabla de hechos que se encuentra en snowflake y github.

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.impute import KNNImputer
import ssl

In [21]:
ssl._create_default_https_context = ssl._create_unverified_context
df=pd.read_csv('https://raw.githubusercontent.com/grupohenryds03/esperanza_vida/main/datasets/EV.csv') # se importa el csv crudo

In [22]:
df.drop_duplicates(inplace = True) # eliminamos las filas duplicadas
indicadores = df['ID_INDICADOR'].unique()
indicadores.sort()
Ind_out = [12] # sacamos el indicador 12 tambien por la redundancia en indicadores
for i in indicadores:
    x = (df[df['ID_INDICADOR'] == i].VALOR.isnull().sum()/len(df[df['ID_INDICADOR'] == i])) * 100
    if x > 20:
        Ind_out.append(i)
for i in Ind_out:
    df = df[df['ID_INDICADOR'] != i] # Sacamos los indicadores dentro de Ind_Out

#-----------------------------------------------------

lista_pais=df['ID_PAIS']
lista_income=df['ID_INCOME']

#-----------------------------------------------------
label_encoder = preprocessing.LabelEncoder()
df['ID_PAIS'] = label_encoder.fit_transform(df['ID_PAIS'])
df['ID_INCOME'] = label_encoder.fit_transform(df['ID_INCOME'])

#-----------------------------------------------------
imputer = KNNImputer(n_neighbors=2, weights='distance') #Reemplazamos los Valores faltantes con KNNImputer
after = imputer.fit_transform(df) #Creamos un Data Frame usando 'after' que tiene los datos imputados.
columnas = df.columns.values
df_limpio = pd.DataFrame(after, columns=columnas) 

#-----------------------------------------------------
df_limpio.ID_PAIS=df_limpio.ID_PAIS.astype(int)
df_limpio.ID_INCOME=df_limpio.ID_INCOME.astype(int)
df_limpio.ID_CONTINENTE=df_limpio.ID_CONTINENTE.astype(int)
df_limpio.ANIO=df_limpio.ANIO.astype(int)
df_limpio.ID_INDICADOR=df_limpio.ID_INDICADOR.astype(int)
df_limpio.VALOR=round(df_limpio.VALOR,2)

In [46]:
df_limpio.to_csv('datasets/EV_limpia.csv')

In [41]:
lista=list(lista_pais)

In [43]:
res = {} 
for key in df_limpio.ID_PAIS: 
    for value in lista: 
        res[key] = value 
        lista.remove(value) 
        break 

In [45]:
print(res)

{0: 'THA', 1: 'BGR', 2: 'PAN', 3: 'ISR', 4: 'IND', 5: 'NGA', 6: 'JPN', 7: 'FRA', 8: 'AUS', 9: 'URY', 10: 'MYS', 11: 'CRI', 12: 'BRA', 13: 'BOL', 14: 'LBY', 15: 'USA', 16: 'ITA', 17: 'SWE', 18: 'CHL', 19: 'GBR', 20: 'NOR', 21: 'IDN', 22: 'ESP', 23: 'MAR', 24: 'GRC', 25: 'TUR', 26: 'PER', 27: 'CHN', 28: 'SAU', 29: 'CAN', 30: 'GRC', 31: 'EGY', 32: 'DEU', 33: 'MEX', 34: 'KOR', 35: 'ZAF', 36: 'RUS'}
